In [1]:
import os 

import pandas as pd

DATA_DIR = "../data"
RESULTS_DIR = os.path.join(DATA_DIR, "results")

In [2]:
results_df = pd.concat([
    pd.read_csv(os.path.join(RESULTS_DIR, f)).assign(cv_split=f) for f in ["ID_k_fold_HFD.csv", "centre_k_fold_HFD.csv", "centre_virtual_k_fold_HFD.csv"]
], ignore_index=True)

In [3]:
grouped_results_df = pd.concat([
    results_df.groupby(["imputer", "cv_split", "Model", "attr_group", "target"])[["AUC", "Accuracy", "Sensitivity (Recall)", "Specificity"]].mean(),
    results_df.groupby(["imputer", "cv_split", "Model", "attr_group", "target"])[["n_positive", "n_total"]].sum()
], axis=1).reset_index().drop("imputer", axis=1)

In [4]:
grouped_results_df["target"] = grouped_results_df["target"].str.lstrip("Dia_")
grouped_results_df = grouped_results_df.query("target == 'HFD'")
grouped_results_df

,cv_split,Model,attr_group,target,AUC,Accuracy,Sensitivity (Recall),Specificity,n_positive,n_total
0,ID_k_fold_HFD.csv,LightGBM,expert,HFD,0.908186,0.827623,0.842888,0.804032,6829.0,11266.0
1,ID_k_fold_HFD.csv,RandomForest,expert,HFD,0.902377,0.824251,0.831909,0.812293,6829.0,11266.0
2,ID_k_fold_HFD.csv,XGBoost,expert,HFD,0.906891,0.824516,0.839962,0.800632,6829.0,11266.0
3,centre_k_fold_HFD.csv,LightGBM,expert,HFD,0.657763,0.696860,0.866319,0.317635,6829.0,11266.0
4,centre_k_fold_HFD.csv,RandomForest,expert,HFD,0.630806,0.623261,0.822638,0.278340,6829.0,11266.0
5,centre_k_fold_HFD.csv,XGBoost,expert,HFD,0.604737,0.709062,0.983993,0.000326,6829.0,11266.0
6,centre_virtual_k_fold_HFD.csv,LightGBM,expert,HFD,0.651157,0.668600,0.763949,0.470230,6829.0,11266.0
7,centre_virtual_k_fold_HFD.csv,RandomForest,expert,HFD,0.639150,0.638409,0.721529,0.475909,6829.0,11266.0
8,centre_virtual_k_fold_HFD.csv,XGBoost,expert,HFD,0.626962,0.728480,0.995670,0.030641,6829.0,11266.0


In [5]:
best_models_df = grouped_results_df.groupby(["cv_split", "attr_group", "target"]).apply(lambda x: x.loc[x["AUC"].idxmax()])

C:\Users\borut.flis\AppData\Local\Temp\ipykernel_23532\1445007316.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  best_models_df = grouped_results_df.groupby(["cv_split", "attr_group", "target"]).apply(lambda x: x.loc[x["AUC"].idxmax()])


In [6]:
best_models_df

,,,cv_split,Model,attr_group,target,AUC,Accuracy,Sensitivity (Recall),Specificity,n_positive,n_total
cv_split,attr_group,target,,,,,,,,,,
ID_k_fold_HFD.csv,expert,HFD,ID_k_fold_HFD.csv,LightGBM,expert,HFD,0.908186,0.827623,0.842888,0.804032,6829.0,11266.0
centre_k_fold_HFD.csv,expert,HFD,centre_k_fold_HFD.csv,LightGBM,expert,HFD,0.657763,0.696860,0.866319,0.317635,6829.0,11266.0
centre_virtual_k_fold_HFD.csv,expert,HFD,centre_virtual_k_fold_HFD.csv,LightGBM,expert,HFD,0.651157,0.668600,0.763949,0.470230,6829.0,11266.0


In [12]:
rename_cols = {"Sensitivity (Recall)": "Sensitivity", "AUC": "c-index", "target": "Predicting", "attr_group": "Attributes"}
final_selection = ["cv_split", "Model", "c-index", "Accuracy", "Majority Accuracy", "Sensitivity", "Specificity"]

In [9]:
#compare_attr_df = best_models_df.loc[pd.IndexSlice[:, ["HFD"]], :].iloc[:-1].rename(columns=rename_cols)
best_models_df["Majority Accuracy"] = best_models_df["n_positive"]/ best_models_df["n_total"]

In [13]:
compare_attr_df = best_models_df.rename(columns=rename_cols)
compare_attr_df["cv_split"] = ["by ID", "by centre", "by centre + virtual"]
# compare_attr_df["Attributes"] = ["MICE", "Expert Selection", "Expert + Blood", "Secondary"]
compare_attr_df["Majority Accuracy"] = compare_attr_df["n_positive"]/ compare_attr_df["n_total"]
compare_attr_df.loc[:, compare_attr_df.dtypes.eq("float")] = compare_attr_df.loc[:, compare_attr_df.dtypes.eq("float")].round(3)
#compare_attr_df.loc[:, final_selection].to_csv(os.path.join(RESULTS_DIR, "final_tables", "compare_attrs.csv"), index=None)
compare_attr_df.loc[:, final_selection]

,,,cv_split,Model,c-index,Accuracy,Majority Accuracy,Sensitivity,Specificity
cv_split,attr_group,target,,,,,,,
ID_k_fold_HFD.csv,expert,HFD,by ID,LightGBM,0.908,0.828,0.606,0.843,0.804
centre_k_fold_HFD.csv,expert,HFD,by centre,LightGBM,0.658,0.697,0.606,0.866,0.318
centre_virtual_k_fold_HFD.csv,expert,HFD,by centre + virtual,LightGBM,0.651,0.669,0.606,0.764,0.470


In [ ]:
final_selection = ["Predicting", "Model", "c-index", "Accuracy", "Majority Accuracy", "Sensitivity", "Specificity"]

compare_expert_df = best_models_df.loc[pd.IndexSlice["expert", :], :].rename(columns=rename_cols)
compare_expert_df["Majority Accuracy"] = compare_expert_df["n_positive"]/ compare_expert_df["n_total"]
compare_expert_df["Majority Accuracy"] = compare_expert_df["Majority Accuracy"].where(
    compare_expert_df["Majority Accuracy"].gt(0.5), 1-compare_expert_df["Majority Accuracy"]
)
compare_expert_df.loc[:, compare_expert_df.dtypes.eq("float")] = compare_expert_df.loc[:, compare_expert_df.dtypes.eq("float")].round(3)

compare_expert_df.loc[:, final_selection].to_csv(os.path.join(RESULTS_DIR, "final_tables", "compare_expert.csv"), index=None)
compare_expert_df.loc[:, final_selection]

In [ ]:
final_selection = ["Predicting", "Model", "c-index", "Accuracy", "Majority Accuracy", "Sensitivity", "Specificity"]

compare_expert_df = best_models_df.loc[pd.IndexSlice["expert_blood", :], :].rename(columns=rename_cols)
compare_expert_df["Majority Accuracy"] = compare_expert_df["n_positive"]/ compare_expert_df["n_total"]
compare_expert_df["Majority Accuracy"] = compare_expert_df["Majority Accuracy"].where(
    compare_expert_df["Majority Accuracy"].gt(0.5), 1-compare_expert_df["Majority Accuracy"]
)
compare_expert_df.loc[:, compare_expert_df.dtypes.eq("float")] = compare_expert_df.loc[:, compare_expert_df.dtypes.eq("float")].round(3)

compare_expert_df.loc[:, final_selection].to_csv(os.path.join(RESULTS_DIR, "final_tables", "compare_expert_blood.csv"), index=None)
compare_expert_df.loc[:, final_selection]

In [ ]:
final_selection = ["Predicting", "Model", "c-index", "Accuracy", "Majority Accuracy", "Sensitivity", "Specificity"]

compare_expert_df = best_models_df.loc[[pd.IndexSlice["secondary", "HFD"]], :].rename(columns=rename_cols)
compare_expert_df["Majority Accuracy"] = compare_expert_df["n_positive"]/ compare_expert_df["n_total"]
compare_expert_df["Majority Accuracy"] = compare_expert_df["Majority Accuracy"].where(
    compare_expert_df["Majority Accuracy"].gt(0.5), 1-compare_expert_df["Majority Accuracy"]
)
compare_expert_df.loc[:, compare_expert_df.dtypes.eq("float")] = compare_expert_df.loc[:, compare_expert_df.dtypes.eq("float")].round(3)

compare_expert_df.loc[:, final_selection].to_csv(os.path.join(RESULTS_DIR, "final_tables", "compare_secondary.csv"), index=None)
compare_expert_df.loc[:, final_selection]